# Feature Manipulation in Pandas

This is a companion notebook for videos part of unit 2 for the data mining course.

Here let's look at a different dataset that will allow us to really dive into some meaningful statistics. This data set is publically available, but is also part of a Kaggle competition.

You can get the data from here: https://www.kaggle.com/c/titanic-gettingStarted 

There are lots of iPython notebooks for looking at the Titanic data. Check them out and see you like any!

In [1]:
# from IPython.display import YouTubeVideo
# YouTubeVideo("9xoqXVjBEF8")

In [6]:
# load the Titantic dataset
import pandas as pd
import numpy as np

df = pd.read_csv('data/titanic.csv')

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
print (df.dtypes)
print (df.info())

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None


In [19]:
# the percentage of individuals that died on the titanic
len(df[df.Survived==0])/len(df)*100.0

61.61616161616161

In [26]:
# Let's aggregate by class and count survival rates
df_grouped = df.groupby(by='Pclass')
print (df_grouped.Survived.sum())
print ('---------------------------------------')
print (df_grouped.Survived.count())
print ('---------------------------------------')
print (df_grouped.Survived.sum() / df_grouped.Survived.count())

Pclass
1    136
2     87
3    119
Name: Survived, dtype: int64
---------------------------------------
Pclass
1    216
2    184
3    491
Name: Survived, dtype: int64
---------------------------------------
Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64


## Feature Discretization

In [32]:
# let's break up the age variable
df['age_range'] = pd.cut(df.Age,[0,16,65,1e6],3,labels=['child','adult','senior']) # this creates a new variable
df.age_range.describe()

count       714
unique        3
top       adult
freq        606
Name: age_range, dtype: object

In [39]:
# now let's group with the new variable
df_grouped = df.groupby(by=['Pclass','age_range'])
print ("Percentage of survivors in each group:")
print (df_grouped.Survived.sum() / df_grouped.Survived.count()* 100.0)

Percentage of survivors in each group:
Pclass  age_range
1       child        88.888889
        adult        65.317919
        senior       25.000000
2       child        90.476190
        adult        42.666667
        senior        0.000000
3       child        40.000000
        adult        20.141343
        senior        0.000000
Name: Survived, dtype: float64


In [48]:
# Let's clean the dataset a little before moving on

# 1. Remove attributes that just aren't useful for us
for col in ['PassengerId','Name','Cabin','Ticket']:
    if col in df:
        del df[col]
        
# 2. Impute some missing values, grouped by their Pclass and SibSp numbers
df_grouped = df.groupby(by=['Pclass','SibSp'])
print (df_grouped.describe())

                Age                                                         \
              count       mean        std    min    25%   50%    75%   max   
Pclass SibSp                                                                 
1      0      113.0  39.181416  14.844591   4.00  28.00  37.0  50.00  80.0   
       1       65.0  37.414154  14.690355   0.92  28.00  38.0  49.00  70.0   
       2        5.0  37.200000  16.513631  18.00  21.00  44.0  50.00  53.0   
       3        3.0  22.000000   2.645751  19.00  21.00  23.0  23.50  24.0   
2      0      109.0  31.934220  13.486824   0.83  23.00  30.0  39.00  70.0   
       1       55.0  27.363636  14.609811   0.67  21.50  29.0  35.00  60.0   
       2        8.0  19.125000  10.776131   1.00  16.75  23.5  24.00  32.0   
       3        1.0  30.000000        NaN  30.00  30.00  30.0  30.00  30.0   
3      0      249.0  27.630201  11.501377   0.42  20.00  26.0  33.00  74.0   
       1       63.0  24.912698  11.439018   1.00  17.50  25.0  3

In [53]:
# now use this grouping to fill the data set in each group, then transform back
df_imputed = df_grouped.transform(lambda grp: grp.fillna(grp.median()))
df_imputed[['Pclass','SibSp','Sex','Embarked']] = df[['Pclass','SibSp','Sex','Embarked']]

print (df_imputed.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Survived    891 non-null int64
Age         884 non-null float64
Parch       891 non-null int64
Fare        891 non-null float64
Pclass      891 non-null int64
SibSp       891 non-null int64
Sex         891 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB
None


/Users/davidstroud/anaconda3/envs/snakes/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:990: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [57]:
# 3. Computed discrete features again now with the newest values
df_imputed['age_range'] = pd.cut(df_imputed.Age,[0,16,65,1e6],3,labels=['child','adult','senior']) # this creates a new variable

print (df_imputed.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
Survived     891 non-null int64
Age          884 non-null float64
Parch        891 non-null int64
Fare         891 non-null float64
Pclass       891 non-null int64
SibSp        891 non-null int64
Sex          891 non-null object
Embarked     889 non-null object
age_range    884 non-null category
dtypes: category(1), float64(2), int64(4), object(2)
memory usage: 56.7+ KB
None


In [59]:
# 4. Drop the rows that still have missing values after grouped imputation
df_imputed.dropna(inplace=True)

# 5. Rearrange the columns
df_imputed = df_imputed[['Survived','Age','age_range','Sex','Parch','SibSp','Pclass','Fare','Embarked']]

print (df_imputed.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 882 entries, 0 to 890
Data columns (total 9 columns):
Survived     882 non-null int64
Age          882 non-null float64
age_range    882 non-null category
Sex          882 non-null object
Parch        882 non-null int64
SibSp        882 non-null int64
Pclass       882 non-null int64
Fare         882 non-null float64
Embarked     882 non-null object
dtypes: category(1), float64(2), int64(4), object(2)
memory usage: 63.0+ KB
None


In [63]:
# did the statistics of the survival group change much? Let's check by repeating the previous grouped calculations
df_grouped = df_imputed.groupby(by=['Pclass', 'age_range'])
print ("Percentage of survivors in each group, with imputed values:")
print (df_grouped.Survived.sum() / df_grouped.Survived.count() * 100.0)

Percentage of survivors in each group, with imputed values:
Pclass  age_range
1       child        88.888889
        adult        62.189055
        senior       25.000000
2       child        90.476190
        adult        42.236025
        senior        0.000000
3       child        37.837838
        adult        22.303922
        senior        0.000000
Name: Survived, dtype: float64
